In [1]:
from dnn.utils.dataLoader import dataLoader
from dnn.utils.yolo_encoder import yoloEncoder
from dnn.parameters import PARAM
from dnn.data.augmentation.detection import DetectionAugmentor
from dnn.data.augmentation.image import ImageAugmentor
from dnn.model.yolov3_tiny import YoloV3Tiny, YoloLoss
from dnn.utils.mem import tf_set_memory_growth
tf_set_memory_growth()

import tensorflow as tf
import numpy as np

In [2]:
size = PARAM['size']
anchors = PARAM['yolo_tiny_anchors']
anchor_masks = PARAM['yolo_tiny_anchor_masks']
num_cls = PARAM['class_number']
train_dir = PARAM['train']
val_dir = PARAM['val']
lr = PARAM['learning_rate']
epoch = PARAM['epoch']
batch_size = PARAM['batch_size']

In [3]:
# check loader
train_loader = dataLoader(train=True)
train = train_loader(train_dir)
train = train.shuffle(buffer_size = 512)
train

<ShuffleDataset shapes: ((None, None, 3), (None, 5)), types: (tf.uint8, tf.float32)>

In [4]:
val_loader = dataLoader(train=False)
val = val_loader(val_dir)
val = val.shuffle(buffer_size = 512)
val

<ShuffleDataset shapes: ((None, None, 3), (None, 5)), types: (tf.uint8, tf.float32)>

In [5]:
# check encoder
encoder = yoloEncoder()
train = encoder(train)
print(train)
train = train.batch(8)
train = train.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
print(train)
val = encoder(val)
val = val.batch(8)
val = val.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected indent (<unknown>, line 31)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected indent (<unknown>, line 31)
<MapDataset shapes: ((None, None, 3), ((13, 13, 3, 6), (26, 26, 3, 6))), types: (tf.uint8, (tf.float32, tf.float32))>
<PrefetchDataset shapes: ((None, None, None, 3), ((None, 13, 13, 3, 6), (None, 26, 26, 3, 6))), types: (tf.uint8, (tf.float32, tf.float32))>
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected indent (<unknown>, line 31)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOS

In [6]:
# check model and loss
model = YoloV3Tiny(size, classes=num_cls, training=True)
loss = [YoloLoss(anchors[mask] / size, classes=num_cls) for mask in np.array(anchor_masks, dtype=np.int8)]
optimizer = tf.keras.optimizers.Adam(lr = lr)
print(model.summary())
print(loss)
print(optimizer)

Model: "yolov3_tiny"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 416, 416, 3) 0                                            
__________________________________________________________________________________________________
yolo_darknet (Model)            multiple             6298480     input[0][0]                      
__________________________________________________________________________________________________
yolo_conv_0 (Model)             (None, 13, 13, 256)  263168      yolo_darknet[1][1]               
__________________________________________________________________________________________________
yolo_conv_1 (Model)             (None, 26, 26, 384)  33280       yolo_conv_0[1][0]                
                                                                 yolo_darknet[1][0]     

Some how this eager training has dtype conversion error but kereas.model.train() works fine

In [7]:
# Eager training
avg_loss = tf.keras.metrics.Mean('loss', dtype=tf.float32)
avg_val_loss = tf.keras.metrics.Mean('val_loss', dtype=tf.float32)
for epoch in range(1, epoch + 1):
    for batch, (images, labels) in enumerate(train):
        with tf.GradientTape() as tape:
            outputs = model(images, training=True)
            regularization_loss = tf.reduce_sum(model.losses)
            pred_loss = []
            for output, label, loss_fn in zip(outputs, labels, loss):
                pred_loss.append(loss_fn(label, output))
            total_loss = tf.reduce_sum(pred_loss) + regularization_loss

        grads = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(
            zip(grads, model.trainable_variables))

        print("Epoch {}.{}, Train Loss: {}, {}".format(
            epoch, batch, total_loss.numpy(),
            list(map(lambda x: np.sum(x.numpy()), pred_loss))))
        avg_loss.update_state(total_loss)
        model.save(PARAM['save_dir'])

    # for batch, (images, labels) in enumerate(val):
    #     outputs = model(images)
    #     regularization_loss = tf.reduce_sum(model.losses)
    #     pred_loss = []
    #     for output, label, loss_fn in zip(outputs, labels, loss):
    #         pred_loss.append(loss_fn(label, output))
    #     total_loss = tf.reduce_sum(pred_loss) + regularization_loss

    #     print("Epoch {}.{}, Validation Loss: {}, {}".format(
    #         epoch, batch, total_loss.numpy(),
    #         list(map(lambda x: np.sum(x.numpy()), pred_loss))))
    #     avg_val_loss.update_state(total_loss)
        break # stop at first batch

    print("Epoch {} Summary, train: {}, val: {}".format(
        epoch,
        avg_loss.result().numpy(),
        avg_val_loss.result().numpy()))

    avg_loss.reset_states()
    avg_val_loss.reset_states()
    break # stop at first epoch

ValueError: Incompatible type conversion requested to type 'uint8' for variable of type 'float32'